<a href="https://colab.research.google.com/github/gaurav-hegde/TwitterSentimentAnalysis/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Kaggle Library

Here we install the Kaggle library to access the dataset through the API.

In [3]:
! pip install kaggle

# Linking Kaggle API using JSON File

Here we configure the path of the Kaggle.json file that contains information about our Kaggle account.

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


#Import Twitter Sentiment Dataset

Using the API we fetch the data from Kaggle.

In [5]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 94% 76.0M/80.9M [00:00<00:00, 99.0MB/s]
100% 80.9M/80.9M [00:00<00:00, 91.0MB/s]


# Extract the compressed file

In [6]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()

# Import Dependencies

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

#printing stopwords in english to ignore
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data Processing

Loading the data into a pandas dataframe from a csv file

In [9]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

#check the shape of the dataset imported
twitter_data.shape

(1599999, 6)

In [10]:
#printing the first 10 rows to gain an understanding of the dataframe
twitter_data.head(10)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [11]:
# naming the column and reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding = 'ISO-8859-1')
twitter_data.shape


(1600000, 6)

In [12]:
twitter_data.head(10)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


# Checking for and replacing missing data

In [13]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

There are no missing values hence we do not have to replace anything

# Checking the distribution of the target column

How many negative (0) , positive tweets (4) there are.

In [14]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

The distribution is even among the positive and negative tweets, hence we do not need to perform any up sampling or down sampling

# Convert the labels

Converting "4" to "1"

In [15]:
twitter_data.replace({'target': {4:1}}, inplace = True)

#checking
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Now we have:

'0' denoting Negative Tweets

'1' denoting Positive Tweets

# Stemming

Process of reducing a word to its root word to make the processing simpler.

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [18]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head(10)

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug,need hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...,loltrish hey long time see ye rain bit bit lol...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it,tatiana k nope
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?,twittera que muera


In [20]:
#To finish data preparation we separate the data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values
print("The data has been separated")

The data has been separated


# Splitting the data

We split the data into training and testing data, this helps us test the accuracy

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [22]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


# Converting the textual data to numerical data

In [23]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [24]:
print(X_train)

  (0, 124681)	0.2233028779907877
  (0, 121435)	0.39810883986331075
  (0, 314728)	0.2308571701707569
  (0, 386282)	0.25974582755604547
  (0, 316069)	0.3732457679700824
  (0, 34505)	0.30859726759816414
  (0, 426283)	0.23555722102790488
  (0, 104687)	0.2995872941437884
  (0, 67715)	0.5395348184170137
  (1, 136253)	0.38440478995211175
  (1, 20576)	0.3427401384251102
  (1, 375793)	0.29628997983279415
  (1, 31038)	0.23427079601609127
  (1, 78765)	0.25448231927322507
  (1, 315345)	0.32546016832481783
  (1, 169281)	0.24680302664271736
  (1, 253068)	0.6004117678811551
  (2, 140623)	0.40504214529827987
  (2, 430896)	0.9142980151636375
  (3, 152428)	0.2932514710776126
  (3, 59849)	0.30862764547479665
  (3, 321025)	0.39613558183962677
  (3, 376404)	0.21350238502643795
  (3, 384883)	0.3132809515432871
  (3, 241733)	0.16739010959880954
  :	:
  (1279994, 443158)	0.37837076919933976
  (1279995, 401605)	0.45232272866924816
  (1279995, 170296)	0.36093372183655587
  (1279995, 94606)	0.3456801780805565
  

# Training the Logistic Regression Model

In [25]:
model = LogisticRegression(max_iter = 1000)

In [26]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation

Using Accuracy Score

In [27]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [28]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.81046640625


In [29]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [30]:
print ('Accuracy score on the test data :', test_data_accuracy)

Accuracy score on the test data : 0.7775375


We notice that the accuracy score on the training and test data are very close by and it is not subject to overfitting. Hence we can say that our model runs successfully.

Model Accuracy = 77.8%

# Saving the trained model

Used for deployment

In [31]:
import pickle

In [32]:
filename = 'TwitterSentimentAnalysis_TrainedModel.sav'
pickle.dump(model, open(filename, 'wb'))

# Using saved model for future predictions


In [33]:
# load the saved model
loaded_model = pickle.load(open('/content/TwitterSentimentAnalysis_TrainedModel.sav', 'rb'))

In [34]:
X_new = X_test[20]
print(Y_test[20])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
